In [1]:
#最低限必要な情報:物件名・住所・間取り・家賃・階数
#検索エリア：中央区（妻との丸の内散歩が趣味）の2K以上の物件、管理費込みで家賃22万円未満

In [2]:
# ライブラリのインポート
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

In [3]:
# URLの設定
base_url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&pc=30&smk=&po1=25&po2=99&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sc=13102&ta=13&cb=0.0&ct=22.0&md=05&md=06&md=07&md=08&md=09&md=10&md=11&md=12&md=13&md=14&ts=1&et=9999999&mb=0&mt=9999999&cn=9999999&fw2={}"

# 初期ページへのリクエストを送信して総ページ数を取得する
response = requests.get(base_url.format(1))
#取得したHTMLを'BeautifulSoup'で解析する
soup_total_page = BeautifulSoup(response.text, 'html.parser')

#最後のPage数を取得する
total_page = int(soup_total_page.find('ol', class_='pagination-parts').find_all('li')[-1].text)

# データ収集用のリスト
data_list = []

# 各ページをループ処理
for page in range(1, total_page + 1):
    # 各ページのURLを生成
    target_url = base_url.format(page)
    # target_urlへのアクセス結果を、変数responseに格納
    response = requests.get(target_url)
    # 負荷を軽減するために1秒スリープ
    time.sleep(1)
    # 現在のページ数と最終ページ数を表示
    print(f"ページ {page}/{total_page} の情報を取得中...")
    # ステータスコードが200以外の場合、ループを終了
    if response.status_code != 200:
        print(f"ページ {page} の取得が終了しました。")
        break
    # 取得結果を解析してsoupに格納
    soup = BeautifulSoup(response.text, 'html.parser')
    # cassetteitemクラスを持つdivタグを全て取得
    property_elements = soup.find_all('div', class_='cassetteitem')
    # 各物件情報をforループで取得する
    for property_element in property_elements:
        # 物件・建物物件を取得し、変数property_detailに格納する
        property_detail = property_element.find('div', class_="cassetteitem-detail")
        # 各部屋の情報を取得し、変数property_tableに格納する
        property_table = property_element.find('table', class_='cassetteitem_other')
        # 物件名を取得し、変数titleに格納する
        title = property_detail.find('div', class_='cassetteitem_content-title').text.strip()
        # 住所を取得し、変数addressに格納する
        address = property_detail.find('li', class_='cassetteitem_detail-col1').text.strip()
        # アクセス情報を取得し、変数accessに格納する
        access = property_detail.find('li', class_='cassetteitem_detail-col2').text.strip()
        # 築年数を取得し、変数ageに格納する
        age = property_detail.find('li', class_='cassetteitem_detail-col3').text.strip()
        # 部屋情報のブロックから、各部屋情報を取得する
        room_elements = property_table.find_all('tr', class_='js-cassette_link')
        # 各部屋情報をforループで取得する
        for room_element in room_elements:
            # 部屋情報の行から、必要な情報を取得する
            floor, price, first_fee, capacity = room_element.find_all('td')[2:6]
            # さらに細かい情報を取得する
            fee, management_fee = price.find_all('li')
            deposit, gratuity = first_fee.find_all('li')
            floor_plan, area = capacity.find_all('li')
            # 取得した情報を辞書に格納する
            property_data = {
                'title': title, # 物件名
                'address': address, # 住所
                'access': access, # アクセス情報
                'age': age, # 築年数
                'floor': floor.text.strip(), # 階数
                'fee': fee.text.strip(), # 家賃
                'management_fee': management_fee.text.strip(), # 管理費
                'deposit': deposit.text.strip(), # 敷金
                'gratuity': gratuity.text.strip(), # 礼金
                'floor_plan': floor_plan.text.strip(), # 間取り
                'area': area.text.strip() # 面積
            }
            # 取得した辞書をリストに追加する
            data_list.append(property_data)

# データフレームの作成
property_df = pd.DataFrame(data_list)

ページ 1/5 の情報を取得中...
ページ 2/5 の情報を取得中...
ページ 3/5 の情報を取得中...
ページ 4/5 の情報を取得中...
ページ 5/5 の情報を取得中...


In [4]:
# CSVファイルに出力
property_df.to_csv('suumo_data.csv', index=False, encoding='utf-8-sig')

print("全ページの情報を取得し、suumo_data.csvに保存しました。")

全ページの情報を取得し、suumo_data.csvに保存しました。
